In [1]:
import numpy as np
from keras.utils import to_categorical
from keras import models
from keras.callbacks import ModelCheckpoint
from keras import layers
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import pickle

Using TensorFlow backend.


In [4]:
DATA = "STATPRO"

In [5]:

train_img_file = pickle.load(open(DATA+"_train_img_filename_for_img_modal_vector.p", "rb"))  # save it into a file named save.p
test_img_file = pickle.load( open(DATA+"_test_img_filename_for_img_modal_vector.p", "rb"))  # save it into a file named save.p
train_img_modal_vector = np.load(DATA+"_train_img_modal_vector.npy")
test_img_modal_vector = np.load(DATA+"_test_img_modal_vector.npy")

FileNotFoundError: [Errno 2] No such file or directory: 'STATPRO_train_img_filename_for_img_modal_vector.p'

In [3]:
y = []
for i in train_img_file:
    if 'pos_' in i:
        y.append(1.0)
    else:
        y.append(0.0)
        
y_test = []
for i in test_img_file:
    if 'pos_' in i:
        y_test.append(1.0)
    else:
        y_test.append(0.0)
        
        

In [4]:
# Input - Layer
model = models.Sequential()
model.add(layers.Dense(50, activation = "relu", input_dim=4096))
# Hidden - Layers
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))

# Output- Layer
model.add(layers.Dense(1, activation = "sigmoid"))

model.summary()
checkpoint = ModelCheckpoint(DATA+'model-best.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')  

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

results = model.fit( train_img_modal_vector,y, epochs= 10, batch_size = 64,validation_data=(test_img_modal_vector,y_test),callbacks=[checkpoint])



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                204850    
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 51        
Total params: 210,001
Trainable params: 210,001
Non-trainable params: 0
_________________________________________________________________
Trai

In [5]:
predicted =  model.predict(test_img_modal_vector)
for i in range(len(predicted)):
    if predicted[i] >= 0.5:
        predicted[i] = 1.0
    else :
        predicted[i] = 0.0

In [8]:
print("fscore",f1_score(y_test, predicted, average="macro"))
print("precision",precision_score(y_test, predicted, average="macro"))
print("recall",recall_score(y_test, predicted, average="macro"))    

x = classification_report(y_test,predicted)
print(x)

fscore 0.725587987086016
precision 0.7265787905868335
recall 0.7269598535195985
             precision    recall  f1-score   support

        0.0       0.69      0.75      0.72       365
        1.0       0.76      0.70      0.73       404

avg / total       0.73      0.73      0.73       769



In [10]:
model.save(DATA+'_only_image_dnn_model')